In [1]:
import gensim
import numpy as np
import string
import tensorflow as tf
import tensorflow.contrib.slim as slim
import Placeholders
import datetime
import random as random

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
n_hidden2 = 50
n_classes = 2
n_examples = 100
support_size = 50

def one_hot(vec, vals = n_classes):
    n = vec.shape[1]
    out = np.zeros((n, vals))
    out[np.arange(n), vec] = 1
    return out


In [3]:
def get_cluster_based_labels1(fc_layer_representation, y, c):
    indexed_centroids = tf.matmul(y,c)
    print(indexed_centroids)
    x_distances = tf.reduce_sum(tf.exp(-(tf.square(fc_layer_representation - indexed_centroids))),1)
    sum_x_distances = tf.log(tf.reduce_sum(x_distances))
    logits = tf.divide(x_distances, sum_x_distances)
    return logits

def get_cluster_based_labels_revised(fc_layer_representation, y, c):
    #centroids_0 = tf.matmul(tf.ones(fc_layer_representation.get_shape()[0]),c[0])
    #centroids_1 = tf.matmul(tf.ones(fc_layer_representation.get_shape()[0]),c[1])
    centroids_0 = c[0]
    centroids_1 = c[1]
    centroid_0_differences  = tf.reduce_sum(tf.exp(-(tf.square(fc_layer_representation - centroids_0))),1) #[n x 1]
    centroid_1_differences  = tf.reduce_sum(tf.exp(-(tf.square(fc_layer_representation - centroids_1))),1) #[n x 1]
    centroid_0_sum = tf.reduce_sum(centroid_0_differences)
    centroid_1_sum = tf.reduce_sum(centroid_1_differences)
    centroid_0_differences = tf.divide(centroid_0_differences, centroid_0_sum)
    centroid_1_differences = tf.divide(centroid_1_differences, centroid_1_sum)
    #print(indexed_centroids)
    #x_distances = tf.exp(-(tf.square(fc_layer_representation - indexed_centroids)))
    #sum_x_distances = tf.log(tf.reduce_sum(x_distances))
    #logits = tf.divide(x_distances, sum_x_distances)
    #logits = tf.nn.softmax(-tf.stack([centroid_0_differences, centroid_1_differences],1))
    logits = tf.stack([centroid_0_differences, centroid_1_differences],1)
    logits_sum = tf.reduce_sum(logits,1)
    final_logits = logits/tf.reshape(logits_sum, (-1, 1)) 
    return final_logits

In [4]:
X_0 = tf.random_normal([n_examples, n_hidden2], mean=10.0, stddev=10.0)
X_1 = tf.random_normal([n_examples, n_hidden2], mean=30.0, stddev=10.0)
X = tf.reshape(tf.stack([X_0, X_1],0),[n_examples*2,n_hidden2])

y_0 = tf.reshape(tf.fill([n_examples, n_classes - 1],0),[-1,n_classes - 1])
y_1 = tf.reshape(tf.fill([n_examples, n_classes - 1],1),[-1,n_classes - 1])
depth = n_classes 
#y = (tf.one_hot(tf.concat([y_0, y_1],0),depth))
y = tf.reshape(tf.one_hot(tf.concat([y_0, y_1],0),n_classes),[n_examples*2,n_classes])

C_0 = tf.reduce_mean(X_0[:support_size],0)
C_1 = tf.reduce_mean(X_1[:support_size],0)
C = tf.stack([C_0,C_1],0)

logits = get_cluster_based_labels_revised(X,y,C)



In [5]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    print("X:",sess.run(tf.shape(X)))
    print("C:",sess.run(tf.shape(C)))
    print("Y:",sess.run(tf.shape(y)))
    logits_ = np.array(sess.run(logits))
    print("Logits Shape:",logits_.shape)
    print(logits_)
    #print("Logits:",logits_[1:3,:])
    #print("Logits:",logits_[101:103,:])
    #print(logits)
    #the_logits = sess.run(logits)
    

X: [200  50]
C: [ 2 50]
Y: [200   2]
Logits Shape: (200, 2)
[[  9.99990940e-01   9.10016479e-06]
 [  9.99968827e-01   3.11431977e-05]
 [  8.92408907e-01   1.07591063e-01]
 [  9.45311785e-01   5.46881706e-02]
 [  8.27867329e-01   1.72132641e-01]
 [  4.31082278e-01   5.68917751e-01]
 [  8.77179742e-01   1.22820266e-01]
 [  9.89327133e-01   1.06728338e-02]
 [  7.11540878e-01   2.88459152e-01]
 [  7.73614466e-01   2.26385608e-01]
 [  8.60625982e-01   1.39374048e-01]
 [  9.99994278e-01   5.72730914e-06]
 [  8.21062922e-01   1.78937092e-01]
 [  7.36152470e-01   2.63847470e-01]
 [  8.80942166e-01   1.19057819e-01]
 [  9.44131732e-01   5.58683053e-02]
 [  1.51011199e-01   8.48988831e-01]
 [  1.00000000e+00   3.15422166e-17]
 [  9.99994457e-01   5.48998696e-06]
 [  9.99745727e-01   2.54298560e-04]
 [  7.51416802e-01   2.48583198e-01]
 [  8.49991083e-01   1.50008932e-01]
 [  9.97337937e-01   2.66208057e-03]
 [  9.55800831e-01   4.41991091e-02]
 [  8.50441635e-01   1.49558350e-01]
 [  7.66377032e